# Introduction to Prompt Engineering
Prompt engineering is the process of designing and optimizing prompts for natural language processing tasks. It involves selecting the right prompts, tuning their parameters, and evaluating their performance. Prompt engineering is crucial for achieving high accuracy and efficiency in NLP models. In this section, we will explore the basics of prompt engineering.

### Exercise 1: Tokenization
Explore Tokenization using tiktoken, an open-source fast tokenizer from OpenAI
See [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) for more examples.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Exercise 2: Validate OpenAI API Key Setup

Run the code below to verify that your OpenAI endpoint is set up correctly. The code just tries a simple basic prompt and validates the completion. Input `oh say can you see` should complete along the lines of `by the dawn's early light..`


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated with Langchain for direct interaction
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage # For creating a user message
from dotenv import load_dotenv
load_dotenv()

# Ensure AZURE_OPENAI_ENDPOINT is set in your .env file
# e.g., AZURE_OPENAI_ENDPOINT=https://YOUR_RESOURCE_NAME.openai.azure.com/

llm = AzureChatOpenAI(
  azure_deployment=os.environ['AZURE_OPENAI_DEPLOYMENT'],
  api_version="2023-05-15", # Check for the latest recommended api_version if needed
  temperature=0, # Controls the randomness of the model's output
  max_tokens=1024,
  # openai_api_key is automatically picked up from AZURE_OPENAI_API_KEY
  # azure_endpoint is automatically picked up from AZURE_OPENAI_ENDPOINT
)

## ---------- Demonstrate Prompt Engineering Directly

### 1. Define the core text or question for the prompt
text_input = f"""
oh say can you see
"""

### 2. Construct the prompt that will be sent to the model
# This is where prompt engineering techniques would be applied.
# For this example, we'll keep the simple formatting.
final_prompt = f"""
Please complete the following phrase:
```{text_input}```
"""

### 3. Create the message object for Langchain
# Langchain's chat models expect a list of messages.
# For a simple user query, we use HumanMessage.
message = HumanMessage(content=final_prompt)

## 4. Run the prompt and get the response
response = llm.invoke([message]) # The core Langchain call

## 5. Print the content of the response
print(response.content)


### Exercise 3: Fabrications
Explore what happens when you ask the LLM to return completions for a prompt about a topic that may not exist, or about topics that it may not know about because it was outside it's pre-trained dataset (more recent). See how the response changes if you try a different prompt, or a different model.

In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = llm.invoke(prompt)
print(response)

### Exercise 4: Instruction Based 
Use the "text" variable to set the primary content 
and the "prompt" variable to provide an instruction related to that primary content.

Here we ask the model to summarize the text for a second-grade student

In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = llm.invoke(prompt)
print(response)

## Langchain Message Types

In Langchain, and generally when working with conversational AI models, messages are used to structure the interaction between a user, the AI, and any initial instructions. Langchain provides specific classes for these:

### SystemMessage:
- **Purpose**: This message is used to set the context, instructions, or persona for the AI model at the beginning of a conversation. It tells the AI how to behave or what role to play.
- **Content**: Typically, it's a string containing instructions. For example, "You are a helpful assistant that translates English to French." or "You are a sarcastic assistant."
- **In the notebook**: "Exercise 5: Complex Prompt" directly demonstrates this concept. The line `{"role": "system", "content": "You are a sarcastic assistant."}` is the equivalent of a SystemMessage. It instructs the OpenAI model to adopt a sarcastic personality for the subsequent interaction. Langchain's SystemMessage formalizes this for its framework.
- **Key takeaway**: It's the initial "priming" or "setup" instruction for the AI. Not all models explicitly support a separate system message, but Langchain handles the adaptation.

### HumanMessage:
- **Purpose**: This represents the input from the human user. It's what the user types or says to the AI.
- **Content**: Usually a string of text, but can also be multi-modal (e.g., an image if the model supports it).
- **In the notebook**:
  - In "Exercise 5: Complex Prompt", `{"role": "user", "content": "Who won the world series in 2020?"}` and `{"role": "user", "content": "Where was it played?"}` are direct examples of user messages.
- **Key takeaway**: It's the direct query or statement from the user to the AI.

### AIMessage:
- **Purpose**: This represents the response generated by the AI model.
- **Content**: Typically the textual answer from the AI. It can also include additional information like requests to call tools (if the AI is an agent that can perform actions).
- **In the notebook**:
  - The `response.choices[0].message.content` you print in "Exercise 2", "Exercise 3", "Exercise 4", and "Exercise 5" is the content of what would be an AIMessage.
  - In "Exercise 5: Complex Prompt", `{"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."}` is an example of an AI's response, which also serves as context for the next user message.
- **Key takeaway**: It's the output or reply from the AI model.

### How They Work Together (Conversation Flow):
Typically, a conversation starts with an optional SystemMessage to set the stage. Then, it alternates between HumanMessage (user asks something) and AIMessage (AI responds). This sequence of messages provides the history and context for the ongoing conversation, allowing the AI to generate more relevant and coherent responses.

Langchain uses these distinct message types to:
- Clearly define the roles in a conversation.
- Maintain a structured history of the interaction.
- Abstract away the specific API requirements of different underlying language models (like OpenAI, Anthropic, etc.), providing a consistent interface.

### Exercise 5: Complex Prompt 
Try a request that has system, user and assistant messages 
System sets assistant context
User & Assistant messages provide multi-turn conversation context

Note how the assistant personality is set to "sarcastic" in the system context. 
Try using a different personality context. Or try a different series of input/output messages

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage # Make sure these are imported

conversation_history = [
    SystemMessage(content="You are a sarcastic assistant."),
    HumanMessage(content="Who won the world series in 2020?"),
    AIMessage(content="Who do you think won? The Los Angeles Dodgers of course."),
    HumanMessage(content="Where was it played?")
]

# Assuming 'llm' is already initialized as AzureChatOpenAI client from previous steps
response_exercise_5 = llm.invoke(conversation_history)

print(response_exercise_5.content)

### Exercise: Explore Your Intuition
The above examples give you patterns that you can use to create new prompts (simple, complex, instruction etc.) - try creating other exercises to explore some of the other ideas we've talked about like examples, cues and more.

H## How to Prompt: Be Explicit About the Desired Structure or Limitations of the Output

### Example Ideas:

- **Prompt**:  
  *"List three benefits of exercise. Provide your answer as a JSON object with a key 'benefits' and a list of strings as the value."*

- **Prompt**:  
  *"Write a poem about the moon. It must be exactly 4 lines long and follow an AABB rhyme scheme."*

- **Prompt**:  
  *"Generate a list of startup ideas in the renewable energy sector. Do not include ideas related to solar panels."*

---

### Expected Output/Observation:

- How strictly does the model adhere to formatting requests and constraints?
- What happens when constraints are very tight or conflicting?
- How is the **temperature** affecting the output?


In [ ]:
## Try your prompt here:
prompt = f"""
Your prompt here
"""
response = llm.invoke(prompt)
print(response)

## Prompt engineering

Prompt engineering is the process of creating prompts that will produce the desired outcome. There's more to prompt engineering than just writing a text prompt. Prompt engineering is not an engineering discipline, it's more a set of techniques that you can apply to get the desired outcome.

### An example of a prompt

Let's take a basic prompt like this one:

> Generate 10 questions on geography.

In this prompt, you are actually applying a set of different prompt techniques.

Let's break this down.

- **Context**, you specify it should be about "geography".
- **Limiting the output**, you want no more than 10 questions.

In [ ]:
response = llm.invoke('Generate 10 questions on geography')
print(response)

### Limitations of simple prompting

You may or may not get the desired outcome. You will get your questions generated, but geography is a big topic and you may not get what you want to due the following reasons:

- **Big topic**, you don't know if it's going to be about countries, capitals, rivers and so on.
- **Format**, what if you wanted the questions to be formatted in a certain way?

As you can see, there's a lot to consider when creating prompts.

So far, we've seen a simple prompt example, but generative AI is capable of much more to help people in a variety of roles and industries. Let's explore some basic techniques next.

### Techniques for prompting

First, we need to understand that prompting is an _emergent_ property of an LLM meaning that this is not a feature that is built into the model but rather something we discover as we use the model.

There are some basic techniques that we can use to prompt an LLM. Let's explore them.

- **Zero-shot prompting**, this is the most basic form of prompting. It's a single prompt requesting a response from the LLM based solely on its training data.
- **Few-shot prompting**, this type of prompting guides the LLM by providing 1 or more examples it can rely on to generate its response.
- **Chain-of-thought**, this type of prompting tells the LLM how to break down a problem into steps.
- **Generated knowledge**, to improve the response of a prompt, you can provide generated facts or knowledge additionally to your prompt.
- **Least to most**, like chain-of-thought, this technique is about breaking down a problem into a series of steps and then ask these steps to be performed in order.
- **Self-refine**, this technique is about critiquing the LLM's output and then asking it to improve.
- **Maieutic prompting**. What you want here is to ensure the LLM answer is correct and you ask it to explain various parts of the answer. This is a form of self-refine.

### Zero-shot prompting

This style of prompting is very simple, it consists of a single prompt. This technique is probably what you're using as you're starting to learn about LLMs. Here's an example:

- Prompt: "What is Algebra?"
- Answer: "Algebra is a branch of mathematics that studies mathematical symbols and the rules for manipulating these symbols."

In [ ]:
# Task: Ask the model to classify the sentiment of a sentence.
zero_shot_prompt_text = "What is Algebra?"

# Create a HumanMessage with the prompt
zero_shot_message = HumanMessage(content=zero_shot_prompt_text)

# Invoke the model
zero_shot_response = llm.invoke([zero_shot_message])

print(f"Prompt: {zero_shot_prompt_text}")
print(f"Response: {zero_shot_response.content}")

### Few-shot prompting

This style of prompting helps the model by providing a few examples along with the request. It consists of a single prompt with additional task-specific data. Here's an example:

- Prompt: "Write a poem in the style of Shakespeare. Here are a few examples of Shakespearean sonnets.:
  Sonnet 18: 'Shall I compare thee to a summer's day? Thou art more lovely and more temperate...'
  Sonnet 116: 'Let me not to the marriage of true minds Admit impediments. Love is not love Which alters when it alteration finds...'
  Sonnet 132: 'Thine eyes I love, and they, as pitying me, Knowing thy heart torment me with disdain,...'
  Now, write a sonnet about the beauty of the moon."
- Answer: "Upon the sky, the moon doth softly gleam, In silv'ry light that casts its gentle grace,..."

Examples provide the LLM with the context, format or style of the desired output. They help the model understand the specific task and generate more accurate and relevant responses.

In [ ]:
# The examples of Shakespearean sonnets
sonnet_18 = "Sonnet 18: 'Shall I compare thee to a summer's day? Thou art more lovely and more temperate...'"
sonnet_116 = "Sonnet 116: 'Let me not to the marriage of true minds Admit impediments. Love is not love Which alters when it alteration finds...'"
sonnet_132 = "Sonnet 132: 'Thine eyes I love, and they, as pitying me, Knowing thy heart torment me with disdain,...'"

# The final request
request = "Now, write a sonnet about the beauty of the moon."

# Constructing the messages for Langchain
shakespeare_few_shot_messages = [
    SystemMessage(content="You are a poet who crafts sonnets in the eloquent style of William Shakespeare."),
    HumanMessage(content=f"""
I would like a sonnet in the style of Shakespeare. 
Here are a few examples of Shakespearean sonnets to guide your style:

Example 1:
{sonnet_18}

Example 2:
{sonnet_116}

Example 3:
{sonnet_132}

{request}
""")
]

# Invoke the model
shakespeare_response = llm.invoke(shakespeare_few_shot_messages)

print("Prompt (condensed for brevity):")
print(f"- System: {shakespeare_few_shot_messages[0].content}")
print(f"- Human: Contains examples of Sonnets 18, 116, 132 and the request: '{request}'")
print(f"\nLLM's Response (a sonnet about the moon in Shakespearean style):")
print(shakespeare_response.content)

### Chain-of-thought

Chain-of-thought is a very interesting technique as it's about taking the LLM through a series of steps. The idea is to instruct the LLM in such a way that it understands how to do something. Consider the following example, with and without chain-of-thought:

    - Prompt: "Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"
    - Answer: 5

LLM answers with 5, which is incorrect. Correct answer is 1 apple, given the calculation (5 -3 -2 + 1 = 1).

So how can we teach the LLM to do this correctly?

Let's try chain-of-thought. Applying chain-of-thought means:

1. Give the LLM a similar example.
1. Show the calculation, and how to calculate it correctly.
1. Provide the original prompt.

Here's how:

- Prompt: "Lisa has 7 apples, throws 1 apple, gives 4 apples to Bart and Bart gives one back:
  7 -1 = 6
  6 -4 = 2
  2 +1 = 3  
  Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"
  Answer: 1

Note how we write substantially longer prompts with another example, a calculation and then the original prompt and we arrive at the correct answer 1.

As you can see chain-of-thought is a very powerful technique.


In [ ]:
# --- Few-Shot Prompting with Chain-of-Thought Example ---
print("\n--- Few-Shot CoT: Apple Problem ---")

# We'll provide one example (the "shot") where the thinking process is shown.
# The SystemMessage can help set the stage.
# The HumanMessage asks the first problem.
# The AIMessage provides the answer WITH the chain of thought.
# The final HumanMessage asks the new problem we want the LLM to solve using CoT.

cot_messages = [
    SystemMessage(content="You are a helpful assistant that solves math word problems by showing each step of your reasoning."),
    HumanMessage(content="Lisa has 7 apples, throws 1 apple, gives 4 apples to Bart and Bart gives one back. How many apples does Lisa have?"),
    AIMessage(content="""Okay, let's break this down step-by-step:
1. Lisa starts with 7 apples.
2. She throws 1 apple: 7 - 1 = 6 apples remaining.
3. She gives 4 apples to Bart: 6 - 4 = 2 apples remaining.
4. Bart gives one apple back to Lisa: 2 + 1 = 3 apples.
So, Lisa has 3 apples."""),
    # Now, the actual problem we want the model to solve using the learned CoT pattern:
    HumanMessage(content="Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?")
]

# Invoke the model
cot_response = llm.invoke(cot_messages)

print("Chain-of-Thought Few-Shot Example:")
print(f"System: {cot_messages[0].content}")
print(f"Human (Example Question): {cot_messages[1].content}")
print(f"AI (Example Answer with CoT): \n{cot_messages[2].content}")
print(f"Human (New Question): {cot_messages[3].content}")
print(f"\nLLM's Response (hopefully showing CoT for Alice's problem):")
print(cot_response.content)

### Least-to-most

The idea with Least-to-most prompting is to break down a bigger problem into subproblems. That way, you help guide the LLM on how to "conquer" the bigger problem. A good example could be for data science where you can ask the LLM to divide up a problem like so:

> Prompt: How to perform data science in 5 steps?

With your AI assistant answering with:

1. Collect data
1. Clean data
1. Analyze data
1. Plot data
1. Present data

### Self-refine, critique the results

With generative AIs and LLMs, you can't trust the output. You need to verify it. After all, the LLM is just presenting you what's the next most likely thing to say, not what's correct. Therefore, a good idea is to ask the LLM to critique itself, which leads us to the self-refine technique.

How it works is that you follow the following steps:

1. Initial prompt asking the LLM to solve a problem
1. LLM answers
1. You critique the answer and ask the AI to improve
1. LLM answers again, this time considering the critique and suggest solutions it came up with

You can repeat this process as many times as you want.


## Good practices

There are many practices you can apply to try to get what you want. You will find your own style as you use prompting more and more.

Additionally to the techniques we've covered, there are some good practices to consider when prompting an LLM.

Here are some good practices to consider:

- **Specify context**. Context matters, the more you can specify like domain, topic, etc. the better.
- Limit the output. If you want a specific number of items or a specific length, specify it.
- **Specify both what and how**. Remember to mention both what you want and how you want it, for example "Create a Python Web API with routes products and customers, divide it into 3 files".
- **Use templates**. Often, you will want to enrich your prompts with data from your company. Use templates to do this. Templates can have variables that you replace with actual data.
- **Spell correctly**. LLMs might provide you with a correct response, but if you spell correctly, you will get a better response.

## Exercise

Here's code in Python showing how to build a simple API using Flask:

```python
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def hello():
    name = request.args.get('name', 'World')
    return f'Hello, {name}!'

if __name__ == '__main__':
    app.run()
```

Use an AI assistant like SoGPT and apply the "self-refine" / other prompting technique to improve the code.

## Solution

Please attempt to solve the assignment by adding suitable prompts to the code.

> [!TIP]
> Phrase a prompt to ask it to improve, it's a good idea to limit how many improvements. You can also ask to improve it in a certain way, for example architecture, performance, security, etc.

## Knowledge check

Why would I use chain-of-thought prompting? Show me 1 correct response and 2 incorrect responses.

1. To teach the LLM how to solve a problem.
1. B, To teach the LLM to find errors in code.
1. C, To instruct the LLM to come up with different solutions.

A: 1, because chain-of-thought is about showing the LLM how to solve a problem by providing it with a series of steps, and similar problems and how they were solved.